### **한국어 BERT를 이용한 네이버 영화 리뷰 분류**

In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

#네이버 영화 리뷰 데이터 학습을 위해 train데이터와 test데이터 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")



('ratings_test.txt', <http.client.HTTPMessage at 0x7ee3c935dd50>)

In [ ]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')


In [ ]:
print('훈 련 용 리 뷰 개 수 :',len(train_data)) # 훈 련 용 리 뷰 개 수 출 력
print('테 스 트 용 리 뷰 개 수 :',len(test_data)) # 테 스 트 용 리 뷰 개 수 출 력

훈 련 용 리 뷰 개 수 : 150000
테 스 트 용 리 뷰 개 수 : 50000


In [ ]:
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열 에 서 중복인 내용이 있다면 중복제거
train_data = train_data.dropna(how='any') # Null값 이 존재하는 행 제거
print('훈 련 데 이 터 의 리 뷰 수 :',len(train_data))

훈 련 데 이 터 의 리 뷰 수 : 146182


In [ ]:
test_data = test_data.dropna(how = 'any')
print('테 스 트 데 이 터 의 리 뷰 수 :',len(test_data))
#테스트 데이터에서도 결측값 제거

테 스 트 데 이 터 의 리 뷰 수 : 49997


In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
#사전 학습된 BERT 류의 모델들의 경우 -> BertTokenizer.from_pretrained(‘모델 이름’) 을 넣으면 해당
#모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

In [ ]:
print(tokenizer.tokenize("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '내내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [ ]:
print(tokenizer.encode("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 6404, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [ ]:
print(tokenizer.decode(tokenizer.encode("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역")))

[CLS] 보는 내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]


In [ ]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)

[CLS] : 2
[SEP] : 3


In [ ]:
print(tokenizer.pad_token, ':', tokenizer.pad_token_id)
#패딩 토큰 확인

[PAD] : 0


In [ ]:
#최대 길이 지정 후 패딩
max_seq_len = 128
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화",
max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길 이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길 이 : 128


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
#문장 구분을 위한 세그먼트 인코딩
print([0]*max_seq_len)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
#실제 단어 토큰과 패딩 토큰을 구분하기 위한 어텐션 마스크
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    # input_id는 워 드 임 베 딩 을 위 한 문 장 의 정 수 인 코 딩
    input_id = tokenizer.encode(example, max_length=max_seq_len,pad_to_max_length=True)

    # attention_mask는 실 제 단 어 가 위 치 하 면 1, 패 딩 의 위 치 에 는 0인 시 퀀 스.
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

    # token_type_id은 세 그 먼 트 인 코 딩
    token_type_id = [0] * max_seq_len
    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 146182/146182 [01:54<00:00, 1278.31it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 49997/49997 [00:20<00:00, 2421.61it/s]


In [ ]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]
print('단 어 에 대 한 정 수 인 코 딩 :',input_id)
print('어 텐 션 마 스 크 :',attention_mask)
print('세 그 먼 트 인 코 딩 :',token_type_id)
print('각 인 코 딩 의 길 이 :', len(input_id))
print('정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))
print('레 이 블 :',label)

단 어 에 대 한 정 수 인 코 딩 : [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세 그 먼 트 인 코 딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
max_seq_len = 128
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)#어텐션 마스크 층
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer]) #bert의 출력

In [ ]:
print(outputs[0])
#아래 결과 : 768차원의 벡터가 128개 있다는 의미로 문장 길이 개수만큼의 출력을 얻음

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


In [ ]:
print(outputs[1])


KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


In [ ]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
# TPU 작 동 을 위 한 코 드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
model.fit(train_X, train_y, epochs=1, batch_size=64, validation_split=0.2)
#배치 크기는 64 로 하고 훈련 데이터의 20% 를 검증 데이터로 사용하여 2 에포크 학습

1828/1828 [==============================] - 334s 135ms/step - loss: 0.2835 - accuracy: 0.8798 - val_loss: 0.2400 - val_accuracy: 0.9023


In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)


49/49 [==============================] - 22s 333ms/step - loss: 0.2499 - accuracy: 0.8979
test loss, test acc:  [0.24988645315170288, 0.8978738784790039]


In [ ]:
def sentiment_predict(new_sentence):
  input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

  padding_count = input_id.count(tokenizer.pad_token_id)
  attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
  token_type_id = [0] * max_seq_len

  input_ids = np.array([input_id])
  attention_masks = np.array([attention_mask])
  token_type_ids = np.array([token_type_id])

  encoded_input = [input_ids, attention_masks, token_type_ids]
  score = model.predict(encoded_input)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict("영화 대박 구리다")

1/1 [==============================] - 9s 9s/step
94.76% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict("와 진짜 재밌었다")

1/1 [==============================] - 1s 874ms/step
99.49% 확률로 긍정 리뷰입니다.



### **KoBERT 를 이용한 KorNLI 풀어보기**

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# train data
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", \
                           filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", \
                           filename="snli_1.0_train.ko.tsv")

# validation data
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", \
                           filename="xnli.dev.ko.tsv")
# test data
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", \
                           filename="xnli.test.ko.tsv")

('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x7a409ef2ed40>)

In [5]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3) #훈련데이터
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3) #훈련데이터
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3) #검증 데이터
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3) #테스트 데이터


In [6]:
# 결 합 후 섞 기
train_data = train_snli.append(train_xnli)
train_data = train_data.sample(frac=1)

<ipython-input-6-63396f581218>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_snli.append(train_xnli)


In [7]:
train_data.head()


,sentence1,sentence2,gold_label
380410,두 마리의 개가 물속에서 막대기를 회수하고 있다.,그 물에는 개가 들어 있다.,entailment
156812,주황색 셔츠를 입은 남자가 기둥 뒤에 있는 콘크리트 선반 위에 웅크리고 누워 있다.,사람이 콘크리트 위에 웅크리고 있다.,entailment
49140,제동액 1파인트는 수백 갤런의 물처럼 많은 갤런의 물을 오염시킬 수 있다.,브레이크 유체는 물에 쉽게 희석되기 때문에 강력한 오염물질이 아니다.,contradiction
386592,스웨터를 입은 어린 소년이 풍선을 가지고 논다.,어린 소년은 지쳤다.,neutral
329624,사격장에서 총을 쏘는 연습을 하고 있는 여자.,한 할머니가 총을 쏜다.,neutral


In [8]:
val_data.head()


,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [9]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


In [10]:
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

#증복 샘플 혹은 결측값 제거

In [11]:
# 결 측 값 및 중 복 샘 플 제 거
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)


In [12]:
print('훈 련 용 샘 플 개 수 :',len(train_data))
print('검 증 용 샘 플 개 수 :',len(val_data))
print('테 스 트 용 샘 플 개 수 :',len(test_data))

훈 련 용 샘 플 개 수 : 941814
검 증 용 샘 플 개 수 : 2490
테 스 트 용 샘 플 개 수 : 5010


In [13]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base") #토크나이저 로드

In [14]:
max_seq_len = 128
sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]
print('문 장1 :',sent1)
print('문 장2 :',sent2)

문 장1 : 두 마리의 개가 물속에서 막대기를 회수하고 있다.
문 장2 : 그 물에는 개가 들어 있다.


In [15]:
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len,pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


encoder_plus 는
*   정수 인코딩 결과인 input_ids,
*   두 개의 문장을 구분하기 위한 token_type_ids,
*   어텐션 마스크에 해당하는 attention_mask 를 자동으로 생성

하여 리턴한다.

In [16]:
print(encoding_result['input_ids'])
#정수 인코딩 결과 확인

[2, 864, 4915, 2079, 558, 2116, 1093, 2354, 27135, 23970, 2138, 7883, 19521, 1513, 2062, 18, 3, 636, 1093, 2170, 2259, 558, 2116, 3685, 1513, 2062, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
print(encoding_result['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
print(encoding_result['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []

  # 정수 인코딩, 세그먼트 인코딩, attention mask를 생성
  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)

    #각각 리스트에 append
    input_ids.append(encoding_result['input_ids'])
    attention_masks.append(encoding_result['attention_mask'])
    token_type_ids.append(encoding_result['token_type_ids'])

  #numpy array로 변환
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  return (input_ids, attention_masks, token_type_ids)

In [20]:
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)


  0%|          | 1995/941814 [00:01<14:32, 1076.55it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|          | 9796/941814 [00:10<24:32, 632.95it/s] Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have 

In [21]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]
print('단 어 에 대 한 정 수 인 코 딩 :',input_id)
print('어 텐 션 마 스 크 :',attention_mask)
print('세 그 먼 트 인 코 딩 :',token_type_id)
print('각 인 코 딩 의 길 이 :', len(input_id))
print('정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))

단 어 에 대 한 정 수 인 코 딩 : [    2   864  4915  2079   558  2116  1093  2354 27135 23970  2138  7883
 19521  1513  2062    18     3   636  1093  2170  2259   558  2116  3685
  1513  2062    18     3     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [29]:
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'],max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/2490 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 2490/2490 [00:03<00:00, 717.53it/s]


In [30]:
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]
print('단 어 에 대 한 정 수 인 코 딩 :',input_id)
print('어 텐 션 마 스 크 :',attention_mask)
print('세 그 먼 트 인 코 딩 :',token_type_id)
print('각 인 코 딩 의 길 이 :', len(input_id))
print('정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))

단 어 에 대 한 정 수 인 코 딩 : [    2  3673   636  2116  1041  2371  2062    16     6  4122    16  1535
  1458 10283    18     6     3   636  2259  3741  4942  2116   636  2138
  4105  2223  2155  6000  4122  2170  2318  4117  2138   572  2359  2062
    18     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [22]:
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|██████████| 5010/5010 [00:08<00:00, 584.33it/s]


In [23]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

# 고 유 한 정 수 로 변 환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print('각 레 이 블 과 정 수 :', label_idx)

각 레 이 블 과 정 수 : {'contradiction': 0, 'entailment': 1, 'neutral': 2}


In [24]:
print('변 환 전 :', train_label[:5])
print('변 환 후 :',y_train[:5])

변 환 전 : ['entailment', 'entailment', 'contradiction', 'neutral', 'neutral']
변 환 후 : [1 1 0 2 2]


In [39]:
from transformers import TFBertForSequenceClassification

class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels, kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),activation='softmax',name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
    cls_token = outputs[1]
    prediction = self.classifier(cls_token)
    return prediction

In [26]:
# TPU 작 동 을 위 한 코 드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)


In [41]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base", num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
#early_stopping 사용하여 일찍 학습을 중단할 수 있게 함
early_stopping = EarlyStopping(monitor="val_accuracy",
                               min_delta=0.001,
                               patience=2)
model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data = (X_val, y_val), callbacks = [early_stopping])

### **KoBERT를 이용한 개체명 인식(Named Entity Recognition)**

In [1]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=01769d208c39535b78e61aacb08c595c654f7bd48b517dba62f4fffabfd39c66
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_train_data.csv", filename="ner_train_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_test_data.csv", filename="ner_test_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_label.txt", filename="ner_label.txt")

('ner_label.txt', <http.client.HTTPMessage at 0x7bcab396b5b0>)

In [5]:
train_ner_df = pd.read_csv("ner_train_data.csv")
test_ner_df = pd.read_csv("ner_test_data.csv")


In [6]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [7]:
test_ner_df.head()

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


In [8]:
print("학 습 데 이 터 샘 플 개 수 :", len(train_ner_df))
print("테 스 트 데 이 터 샘 플 개 수 :", len(test_ner_df))

학 습 데 이 터 샘 플 개 수 : 81000
테 스 트 데 이 터 샘 플 개 수 : 9000


In [9]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]


In [10]:
print(train_data_sentence[2])
print(train_data_label[2])


['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


형태소 단위가 아니라 어절 단위 (띄어쓰기 단위) 로 개체명 인식 레이블이 태깅

In [11]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개 체 명 태 깅 정 보 :', labels)


개 체 명 태 깅 정 보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [12]:
#개체명 태깅 정보를 저장한 labels 리스트로부터
#각 개체명 태깅 정보와 정수를 맵핑하는 딕셔너리를 만든다
#tag_to_index -> 개체명 태깅 정보 : key, 정수 : value로 하는 딕셔너리
#index_to_tag -> 정수 : key, 개체명 태깅 정보 : value로 하는 딕셔너리
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [13]:
print(tag_to_index)
print(index_to_tag)

{'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}
{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


In [14]:
tag_size = len(tag_to_index)
print('개 체 명 태 깅 정 보 의 개 수 :',tag_size)


개 체 명 태 깅 정 보 의 개 수 : 29


In [15]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base") #토크 나이저 로드

In [16]:
sent = train_data_sentence[1]
label = train_data_label[1]
print('문 장 :', sent)
print('레 이 블 :',label)
print('레 이 블 의 정 수 인 코 딩 :',[tag_to_index[idx] for idx in label])
print('문 장 의 길 이 :', len(sent))
print('레 이 블 의 길 이 :', len(label))


문 장 : ['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
레 이 블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레 이 블 의 정 수 인 코 딩 : [0, 1, 2, 0, 1, 0, 1]
문 장 의 길 이 : 7
레 이 블 의 길 이 : 7


In [17]:
tokens = []

for one_word in sent:
# 각 단 어 에 대 해 서 서 브 워 드 로 분 리.
# ex) one_word = '쿠 마 리' ===> subword_tokens = ['쿠', '##마 리']
# ex) one_word = '한 동 수 가' ===> subword_tokens = ['한 동', '##수', '##가']
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)

print('BERT 토 크 나 이 저 적 용 후 문 장 :',tokens)
print('레 이 블 :', label)
print('레 이 블 의 정 수 인 코 딩 :',[tag_to_index[idx] for idx in label])
print('문 장 의 길 이 :', len(tokens))
print('레 이 블 의 길 이 :', len(label))


BERT 토 크 나 이 저 적 용 후 문 장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레 이 블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레 이 블 의 정 수 인 코 딩 : [0, 1, 2, 0, 1, 0, 1]
문 장 의 길 이 : 16
레 이 블 의 길 이 : 7


'쿠마리' -> '쿠' , '##마리' : 서브워드 분리 후 레이블 처리는 아래와 같다.
*   뒤의 서브워드에 대해서는 레이블을 주지 않음 : -100을 부여

In [18]:
tokens = []
labels_ids = []

for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]]+ [-100] * (len(subword_tokens) -1))

print('토 큰 화 후 문 장 :',tokens)
print('레 이 블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레 이 블 의 정 수 인 코 딩 :', labels_ids)
print('문 장 의 길 이 :', len(tokens))
print('레 이 블 의 길 이 :', len(labels_ids))

토 큰 화 후 문 장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레 이 블 : ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레 이 블 의 정 수 인 코 딩 : [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
문 장 의 길 이 : 16
레 이 블 의 길 이 : 16


In [19]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,pad_token_id_for_segment=0,pad_token_id_for_label=-100):
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    tokens = []
    labels_ids = []
    for one_word, label_token in zip(example, label):
      # 하 나 의 단 어 에 대 해 서 서 브 워 드 로 토 큰 화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서 브 워 드 중 첫 번 째 서 브 워 드 만 개 체 명 레 이 블 을 부 여 하 고 그 외 에 는-100으 로 채 운 다.
      labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label]* (len(subword_tokens) - 1))

    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

    # [SEP]를 추 가 하 는 코 드
    # 1. 토 큰 화 결 과 의 맨 뒷 부 분 에 [SEP] 토 큰 추 가
    # 2. 레 이 블 에 도 맨 뒷 부 분 에 -100 추 가.
    tokens += [sep_token]
    labels_ids += [pad_token_id_for_label]

    # 정 수 인 코 딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    # 어 텐 션 마 스 크 생 성
    attention_mask = [1] * len(input_id)
    # 정 수 인 코 딩 에 추 가 할 패 딩 길 이 연 산
    padding_count = max_seq_len - len(input_id)
    # 정 수 인 코 딩 , 어 텐 션 마 스 크 에 패 딩 추 가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)
    # 세 그 먼 트 인 코 딩.
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    # 레 이 블 패 딩. (단 , 이 경 우 는 패 딩 토 큰 의 ID가 -100)
    label = labels_ids + ([pad_token_id_for_label] * padding_count)
    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {}vs {}".format(len(token_type_id), max_seq_len)
    assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels


In [20]:
X_train, y_train = convert_examples_to_features(train_data_sentence,train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label,max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:06<00:00, 1420.01it/s]


In [21]:
print('기 존 원 문 :', train_data_sentence[0])
print('기 존 레 이 블 :', train_data_label[0])
print('-' * 50)
print('토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_train[0][0]])
print('토 큰 화 후 레 이 블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[0]])
print('-' * 50)
print('정 수 인 코 딩 결 과 :', X_train[0][0])
print('정 수 인 코 딩 레 이 블 :', y_train[0])


기 존 원 문 : ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기 존 레 이 블 : ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
토 큰 화 후 원 문 : ['정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [22]:
print('세 그 먼 트 인 코 딩 :', X_train[2][0])
print('어 텐 션 마 스 크 :', X_train[1][0])


세 그 먼 트 인 코 딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [23]:
from transformers import TFBertForTokenClassification

class TFBertForTokenClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForTokenClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
    # 전 체 시 퀀 스 에 대 해 서 분 류 해 야 하 므 로 outputs[0]임 에 주 의
    all_output = outputs[0]
    prediction = self.classifier(all_output)
    return prediction

In [24]:
def compute_loss(labels, logits):
  # 다 중 클 래 스 분 류 문 제 에 서 소 프 트 맥 스 함 수 미 사 용 시 from_logits=True로 설 정.
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  # -100의 값 을 가 진 정 수 에 대 해 서 는 오 차 를 반 영 하 지 않 도 록 labels를 수 정.
  active_loss = tf.reshape(labels, (-1,)) != -100
  # activa_loss로 부 터 reduced_logits과 labels를 각 각 얻 는 다.
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  return loss_fn(labels, reduced_logits)


In [25]:
# TPU 작 동 을 위 한 코 드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)


In [26]:
with strategy.scope():
  model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=compute_loss)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [30]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])

        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):
      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

In [31]:
f1_score_report = F1score(X_test, y_test)

In [32]:
model.fit(
    X_train, y_train, epochs=1, batch_size=32,
    callbacks = [f1_score_report]
)


   6/2532 [..............................] - ETA: 3:27 - loss: 0.1607

282/282 [==============================] - 7s 22ms/step
 - f1: 85.91
              precision    recall  f1-score   support

         AFW       0.62      0.63      0.63       394
         ANM       0.79      0.72      0.75       701
         CVL       0.84      0.84      0.84      5758
         DAT       0.91      0.92      0.92      2521
         EVT       0.75      0.78      0.76      1094
         FLD       0.70      0.52      0.60       228
         LOC       0.83      0.88      0.85      2126
         MAT       0.15      0.17      0.16        12
         NUM       0.91      0.93      0.92      5590
         ORG       0.89      0.86      0.87      4086
         PER       0.88      0.90      0.89      4426
         PLT       0.75      0.09      0.16        34
         TIM       0.84      0.92      0.88       314
         TRM       0.72      0.76      0.74      1964

   micro avg       0.85      0.86      0.86     29248
   macro avg       0.76      0.71      0.71     29248
weighted av

In [33]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {}vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks


In [34]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 503.31it/s]


In [35]:
print('기 존 원 문 :', test_data_sentence[0])
print('-' * 50)
print('토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레 이 블 마 스 크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

기 존 원 문 : ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']
--------------------------------------------------
토 큰 화 후 원 문 : ['[CLS]', '라', '##티', '##은', '-', '원', '##윤', '##정', ',', '휘', '##닉스', '##파크', '##클', '##래', '##식', '프로', '##골', '##퍼', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

In [36]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples,max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []
  for i in range(0, len(label_masks)):
    pred_tag = []
    # ex) 모 델 의 예 측 값 디 코 딩 과 정
    # 예 측 값(y_predicted)에 서 레 이 블 마 스 크(label_masks)의 값 이 -100인 동 일 위 치 의값 을 삭 제

    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])
    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)
  return result_list

In [37]:
sent1 = '다은이는 실습이 너무나도 어렵다'
sent2 = '지금은 아이브의 이덜웨이를 들으면서 실습중이다'

test_samples = [sent1, sent2]

result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)
result_list


100%|██████████| 2/2 [00:00<00:00, 1317.10it/s]


1/1 [==============================] - 1s 646ms/step


[[('다은이는', 'O'), ('실습이', 'O'), ('너무나도', 'O'), ('어렵다', 'O')],
 [('지금은', 'O'),
  ('아이브의', 'PER-B'),
  ('이덜웨이를', 'AFW-B'),
  ('들으면서', 'O'),
  ('실습중이다', 'O')]]

### **KoBERT를 이용한 기계 독해**

In [38]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2023-10-09 11:42:51--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   139MB/s    in 0.3s    

2023-10-09 11:42:51 (139 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2023-10-09 11:42:51--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [39]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

print('훈 련 데 이 터 의 본 문 개 수 :', len(train_contexts))
print('훈 련 데 이 터 의 질 문 개 수 :', len(train_questions))
print('훈 련 데 이 터 의 답 변 개 수 :', len(train_answers))
print('테 스 트 데 이 터 의 본 문 개 수 :', len(val_contexts))
print('테 스 트 데 이 터 의 질 문 개 수 :', len(val_questions))
print('테 스 트 데 이 터 의 답 변 개 수 :', len(val_answers))

훈 련 데 이 터 의 본 문 개 수 : 60407
훈 련 데 이 터 의 질 문 개 수 : 60407
훈 련 데 이 터 의 답 변 개 수 : 60407
테 스 트 데 이 터 의 본 문 개 수 : 5774
테 스 트 데 이 터 의 질 문 개 수 : 5774
테 스 트 데 이 터 의 답 변 개 수 : 5774


In [40]:
print('첫 번 째 샘 플 의 본 문')
print('-----------------')
print(train_contexts[0])


첫 번 째 샘 플 의 본 문
-----------------
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [41]:
print('첫 번 째 샘 플 의 질 문')
print('-----------------')
print(train_questions[0])


첫 번 째 샘 플 의 질 문
-----------------
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [42]:
print('첫 번 째 샘 플 의 답 변')
print('-----------------')
print(train_answers[0])


첫 번 째 샘 플 의 답 변
-----------------
{'text': '교향곡', 'answer_start': 54}


In [43]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip()
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [44]:
print('첫 번 째 샘 플 의 답 변')
print('-----------------')
print(train_answers[0])

첫 번 째 샘 플 의 답 변
-----------------
{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


In [45]:
train_contexts[0][54:57]

'교향곡'

In [46]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [47]:
print('첫 번 째 샘 플 의 토 큰 화 결 과 :', train_encodings[0].tokens)

첫 번 째 샘 플 의 토 큰 화 결 과 : ['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현', '##악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '#

In [48]:
print('첫 번 째 샘 플 의 길 이 :', len(train_encodings[0].tokens))

첫 번 째 샘 플 의 길 이 : 512


In [49]:
print('첫 번 째 샘 플 의 정 수 인 코 딩 :', train_encodings[0].ids)

첫 번 째 샘 플 의 정 수 인 코 딩 : [2, 13934, 2236, 2440, 27982, 2259, 21310, 2079, 11994, 3791, 2069, 3790, 1508, 2088, 636, 3800, 2170, 3717, 2052, 9001, 8345, 4642, 2200, 3689, 3657, 2079, 19282, 2069, 1363, 2370, 2259, 936, 2069, 554, 2259, 2062, 18, 1504, 4342, 27982, 2259, 13934, 2196, 2440, 2170, 1195, 23260, 6233, 17370, 2113, 2165, 2069, 809, 1, 3706, 2052, 2181, 8642, 2145, 7334, 2170, 4983, 2371, 4007, 1065, 5917, 2386, 2559, 4443, 2138, 4026, 2259, 11994, 3791, 2079, 15864, 2170, 5487, 2371, 4683, 3605, 18, 3819, 5986, 27135, 1376, 2645, 2203, 2292, 2079, 5872, 2200, 5986, 4152, 2252, 22835, 16706, 2052, 5485, 2205, 2259, 17087, 2079, 19282, 29, 2517, 2069, 881, 2088, 652, 2073, 23404, 2069, 1122, 2886, 13964, 16, 3982, 2052, 9944, 21, 2429, 2170, 11994, 3791, 2079, 1258, 2465, 6233, 24294, 1504, 3967, 2170, 4027, 2052, 5121, 3979, 2069, 18274, 21575, 23548, 575, 2073, 5292, 2085, 7251, 2116, 1415, 2062, 18, 3776, 2079, 942, 2286, 2446, 4196, 2079, 3640, 6509, 636, 2079, 4450, 2170, 1

In [50]:
print('첫 번 째 샘 플 의 어 텐 션 마 스 크 :', train_encodings[0].attention_mask)

첫 번 째 샘 플 의 어 텐 션 마 스 크 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [51]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []

    for i in tqdm(range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
              deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list


In [52]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 149802.13it/s]


In [53]:
print('삭 제 예 정 인 훈 련 샘 플 :', deleting_list_for_train)
print('삭 제 예 정 인 테 스 트 샘 플 :', deleting_list_for_test)


삭 제 예 정 인 훈 련 샘 플 : [711, 726, 728, 729, 761, 765, 767, 768, 805, 2586, 2587, 2722, 2724, 2725, 2731, 3392, 3475, 3478, 3491, 3495, 3498, 3919, 4462, 4465, 4513, 4515, 4565, 4765, 4766, 4772, 4774, 4779, 5334, 6603, 6638, 6639, 6748, 6749, 6750, 6765, 6766, 6771, 6776, 6897, 6898, 6900, 7739, 7741, 9203, 9211, 10880, 11039, 11212, 11727, 11776, 11788, 11789, 11791, 12168, 13708, 13711, 13996, 14460, 14461, 14491, 14724, 14729, 14885, 15764, 15970, 15971, 15973, 15974, 15976, 15977, 15979, 15980, 16080, 17683, 17815, 17828, 18389, 18392, 19045, 19052, 19053, 19195, 19636, 19637, 19638, 19640, 19656, 19761, 19764, 19765, 20614, 20618, 21224, 21243, 21334, 21335, 21338, 21361, 21521, 21522, 22627, 22633, 24003, 24577, 24579, 24580, 24768, 25108, 25176, 25182, 25185, 25186, 25187, 25188, 25448, 25451, 25454, 25457, 25460, 27105, 27112, 27113, 27114, 27159, 27293, 27295, 27555, 27558, 28025, 28438, 28779, 29162, 29189, 29289, 29290, 29855, 31889, 31890, 31891, 31894, 31905, 32050, 32051, 32

In [54]:
print('761번 샘 플 의 질 문 :',train_questions[761])
print('-'*50)
print('761번 샘 플 의 기 존 원 문 :',train_contexts[761])

761번 샘 플 의 질 문 : 소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최종 성적은?
--------------------------------------------------
761번 샘 플 의 기 존 원 문 : 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름(21)은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영(25-강원도청)이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영(24-전북도청)은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트(네덜란드)가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바(체코-4분01초95)와 3위 올가 그라프(러시아-4분03초47)에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트(1분54초09)에게 돌아갔고 동메달은 하를로터 판바이크(1분54초54)가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라(1분56초40)가 차지했다. 김보름은 1분59초78로 21위에 올랐다. 노선영(25-강원도청)

In [55]:
print('761번 샘 플 전 처 리 후 :', tokenizer.decode(train_encodings['input_ids'][761]))


761번 샘 플 전 처 리 후 : [CLS] 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름 ( 21 ) 은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영 ( 25 - 강원도청 ) 이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영 ( 24 - 전북도청 ) 은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트 ( 네덜란드 ) 가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바 ( 체코 - 4분01초95 ) 와 3위 올가 그라프 ( 러시아 - 4분03초47 ) 에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트 ( 1분54초09 ) 에게 돌아갔고 동메달은 하를로터 판바이크 ( 1분54초54 ) 가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라 ( 1분56초40 ) 가 차지했다. 김보름은 1분59초78로 21위에 올랐다. 노선영 ( 25 - 강원도청 ) 은 2분01초07로 29위, 양신영 ( 24 - 전북도청 ) [SEP] 소치 팀추월 파

In [56]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [57]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [58]:
print('-------------삭 제 전 -------------')
print('훈 련 데 이 터 의 샘 플 의 개 수 :', len(train_contexts))
print('테 스 트 데 이 터 의 샘 플 의 개 수 :', len(val_contexts))
print('-------------삭 제 후 -------------')
print('훈 련 데 이 터 의 샘 플 의 개 수 :', len(X_train[0]))
print('테 스 트 데 이 터 의 샘 플 의 개 수 :', len(X_test[0]))


-------------삭 제 전 -------------
훈 련 데 이 터 의 샘 플 의 개 수 : 60407
테 스 트 데 이 터 의 샘 플 의 개 수 : 5774
-------------삭 제 후 -------------
훈 련 데 이 터 의 샘 플 의 개 수 : 60140
테 스 트 데 이 터 의 샘 플 의 개 수 : 5717


In [59]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [60]:
strategy = tf.distribute.TPUStrategy(resolver)

In [61]:
from transformers import TFBertModel

class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        # start_logits = (batch_size, sequence_length,)
        # end_logits = (batch_size, sequence_length,)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [62]:
with strategy.scope():
  model = TFBertForQuestionAnswering("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [63]:
history = model.fit(
    X_train,
    y_train,
    epochs=3,
    verbose=1,
    batch_size=16,
)

Epoch 1/3


3759/3759 [==============================] - 579s 119ms/step - loss: 1.1307 - output_1_loss: 0.5080 - output_2_loss: 0.6226
Epoch 2/3
3759/3759 [==============================] - 445s 118ms/step - loss: 0.6202 - output_1_loss: 0.2654 - output_2_loss: 0.3548
Epoch 3/3
3759/3759 [==============================] - 444s 118ms/step - loss: 0.4507 - output_1_loss: 0.1856 - output_2_loss: 0.2652


In [64]:
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
  print('정답 :',tokenizer.decode(answer_encoded))
  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  answer_encoded = X_test[0][idx][start:end]
  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [65]:
for i in range(0, 100):
  predict_test_data_by_idx(i)


본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 : 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? 
정답 : 1989년 2월 15일
예측 : 1989년 2월 15일
----------------------------------------
본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학

### **BERT의 문장 임베딩을 이용한 한국어 챗봇**

In [68]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d6229da872bc8d4d9813843390828e1585178c8e1d67b72327e4f7f224990d84
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [69]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [70]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')


In [71]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis =1)


In [72]:
def cos_sim(A, B): #코사인 유사도 구하기
  return dot(A, B)/(norm(A)*norm(B))

In [74]:
def return_answer(question):
  embedding = model.encode(question)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'],embedding), axis=1)
  return train_data.loc[train_data['score'].idxmax()]['A']


In [75]:
return_answer('열심히 살려면 어떻게 해야될까')

'일을 몰라서 그런가봐요.'

In [76]:
return_answer('영어공부는 어떻게 해야 돼')

'공부 좋죠.'

In [77]:
return_answer('챗봇을 사용하려면 1개의 질문 당 몇개의 답이 필요할까')

'아마 그럴 거예요.'

In [78]:
return_answer('나 졸려')

'오늘 일찍 주무세요.'